In [1]:
import pandas as pd
import numpy as np
import os; os.chdir('../')
import warnings
from urbansim_templates import modelmanager as mm

import orca
import urbansim_templates

from activitysynth.scripts import models, datasources, variables

warnings.simplefilter('ignore')

Registering model step 'auto_ownership'
Registering model step 'dwell_work'
Registering model step 'TOD_choice'
Registering model step 'work_TOD_choice'
Registering model step 'primary_mode_choice'
Registering model step 'WLCM'


In [2]:
import choicemodels; choicemodels.__version__

'0.2.1'

In [3]:
import urbansim_templates; urbansim_templates.__version__

'0.2.dev0'

### Generate accessibility variables (or load them if they already exist)

Initialize networks

In [4]:
orca.run(['initialize_network_small', 'initialize_network_walk'])

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s


If they exist already:

In [6]:
walk_net_vars = pd.read_csv('./data/walk_net_vars.csv', index_col='osmid')
drive_net_vars = pd.read_csv('./data/drive_net_vars.csv', index_col='osmid')
orca.add_table('nodeswalk', walk_net_vars)
orca.add_table('nodessmall', drive_net_vars)

If generating on-the-fly:

In [10]:
# orca.run(['network_aggregations_small', 'network_aggregations_walk'])

### Simulate workplace location choices

Run the model step

In [8]:
warnings.simplefilter('ignore')
accessibilities_mode = 'compute'
year = 2010
data_mode = 'csv'
output_tables = [
    'parcels', 'buildings', 'jobs', 'persons', 'households',
    'establishments', 'rentals', 'units']
output_bucket = 'urbansim-outputs'
beam_bucket = 'urbansim-beam'
local_data_dir = '/home/data/fall_2018/'
fname_walk = 'walk_net_vars.csv'
fname_drive = 'drive_net_vars.csv'

In [9]:
orca.add_injectable('data_mode', data_mode)

if data_mode == 'h5':
        @orca.injectable('store', cache=True)
        def hdfstore():
            return pd.HDFStore(
                (local_data_dir + "model_data.h5"), mode='r')
        orca.add_injectable('s3_input_data_url', None)
        orca.add_injectable('local_data_dir', None)

elif data_mode == 's3':
        orca.add_injectable('store', None)
        orca.add_injectable(
            's3_input_data_url',
            's3://urbansim-baseyear-inputs/{0}.parquet.gz')
        orca.add_injectable('local_data_dir', None)

elif data_mode == 'csv':
        orca.add_injectable('store', None)
        orca.add_injectable('s3_input_data_url', None)
        orca.add_injectable('local_data_dir', local_data_dir)

In [10]:
orca.run(['wlcm_simulate'])

Running step 'wlcm_simulate'
Iteration 1: 191268 of 3060996 valid choices
Iteration 2: 381754 of 3060996 valid choices
Iteration 3: 571228 of 3060996 valid choices
Iteration 4: 759874 of 3060996 valid choices
Iteration 5: 947307 of 3060996 valid choices
Iteration 6: 1133502 of 3060996 valid choices
Iteration 7: 1317704 of 3060996 valid choices
Iteration 8: 1500236 of 3060996 valid choices
Iteration 9: 1679523 of 3060996 valid choices
Iteration 10: 1854961 of 3060996 valid choices
Iteration 11: 2024888 of 3060996 valid choices
Iteration 12: 2186543 of 3060996 valid choices
Iteration 13: 2334773 of 3060996 valid choices
Iteration 14: 2459129 of 3060996 valid choices
Iteration 15: 2540249 of 3060996 valid choices
Iteration 16: 2573729 of 3060996 valid choices
Iteration 17: 2578044 of 3060996 valid choices
Iteration 18: 2578046 of 3060996 valid choices
No valid alternatives for the remaining choosers
Time to execute step 'wlcm_simulate': 300.98 s
Total time to execute iteration 1 with iter

Query the updated persons table to assess job placement

In [22]:
persons_w_jobs = orca.get_table('persons').to_frame(['job_id', 'worker','work_at_home'])
num_persons_w_jobs = len(persons_w_jobs[~pd.isnull(persons_w_jobs['job_id'])])
wlcm = urbansim_templates.modelmanager.get_step('WLCM')
num_workers = len(persons_w_jobs.query(' and '.join(wlcm.out_chooser_filters)))

In [7]:
print("WLCM placed {0} of {1} workers ({2}%) during simulation.".format(
    num_persons_w_jobs, num_workers, np.round(num_persons_w_jobs/num_workers * 100, 1)))

WLCM placed 2578046 of 3060996 workers (84.2%) during simulation.


### Simulate auto-ownership choices

In [11]:
orca.run(['auto_ownership_simulate'])

Running step 'auto_ownership_simulate'
Time to execute step 'auto_ownership_simulate': 74.51 s
Total time to execute iteration 1 with iteration value None: 74.51 s


In [14]:
households_w_auto_ownership = orca.get_table('households').to_frame()
num_households_w_auto = len(households_w_auto_ownership[~pd.isnull(households_w_auto_ownership['cars'])])

In [15]:
print("Auto ownership assigned to {0} of {1} households ({2}%) during simulation.".format(
    num_households_w_auto, len(households_w_auto_ownership),
    np.round(num_households_w_auto/len(households_w_auto_ownership) * 100, 1)))

Auto ownership assigned to 2677468 of 2677468 households (100.0%) during simulation.


### Simulate time-of-day choices for mandatory trip (commute) departure times

#### a) Assign commuters to depature time chunks

In [12]:
orca.run(['TOD_category_simulate'])

Running step 'TOD_category_simulate'
Time to execute step 'TOD_category_simulate': 43.88 s
Total time to execute iteration 1 with iteration value None: 43.88 s


In [13]:
orca.run(['TOD_dwell_simulate'])

Running step 'TOD_dwell_simulate'
Time to execute step 'TOD_dwell_simulate': 44.39 s
Total time to execute iteration 1 with iteration value None: 44.39 s


#### b) Assign actual departure times based on known distributions

In [14]:
orca.run(['TOD_distribution_simulate'])

Running step 'TOD_distribution_simulate'
Time to execute step 'TOD_distribution_simulate': 1514.15 s
Total time to execute iteration 1 with iteration value None: 1514.15 s


Query the updated persons table to assess TOD chunk assignment

In [21]:
persons_w_TOD2 = orca.get_table('persons').to_frame('HW_ST')
num_persons_w_TOD = len(persons_w_TOD2[~pd.isnull(persons_w_TOD2['HW_ST'])])

In [34]:
print("Departure time assigned to {0} of {1} workers with jobs ({2}%) during simulation.".format(
    num_persons_w_TOD, num_persons_w_jobs, np.round(num_persons_w_TOD/num_persons_w_jobs * 100, 1)))

Departure time assigned to 2502445 of 2578046 workers with jobs (97.1%) during simulation.


### Simulate primary commute mode choice

In [23]:
orca.run(['primary_mode_choice_simulate'])

In [17]:
persons_w_mode = orca.get_table('persons_CHTS_format').to_frame()

In [18]:
num_persons_w_mode = len(persons_w_mode[~pd.isnull(persons_w_mode['primary_commute_mode'])])

In [19]:
print("Primary commute mode assigned to {0} of {1} workers with jobs ({2}%) during simulation.".format(
    num_persons_w_mode, num_persons_w_jobs,
    np.round(num_persons_w_mode/num_persons_w_jobs * 100, 1)))

Primary commute mode assigned to 2578046 of 2578046 workers with jobs (100.0%) during simulation.


### Generate activity plans

Activity plans currently written according to [BEAM](http://beam.lbl.gov/) specification

In [21]:
orca.run(['generate_activity_plans'])

Running step 'generate_activity_plans'
Time to execute step 'generate_activity_plans': 150.72 s
Total time to execute iteration 1 with iteration value None: 150.72 s


Load activity plan data and investigate it

In [22]:
plans = orca.get_table('plans').to_frame()

In [23]:
plans.head()

,personId,planElement,planElementIndex,activityType,x,y,endTime
0,0,activity,1,Home,-121.882,37.3391,6.87786
1,0,leg,2,,,,
2,0,activity,3,Work,-121.886,37.3445,17.9252
3,0,leg,4,,,,
4,0,activity,5,Home,-121.882,37.3391,


In [26]:
activities = plans[plans['planElement'] == 'activity']

In [29]:
activities[pd.isnull(activities['endTime'])].head()

,personId,planElement,planElementIndex,activityType,x,y,endTime
4430145,2502445,activity,1,Home,-122.199,37.7618,NaN
4430147,2502445,activity,3,Work,-121.966,37.7632,NaN
4430150,2502446,activity,1,Home,-122.199,37.7618,NaN
4430152,2502446,activity,3,Work,-122.257,37.8249,NaN
4430155,2502451,activity,1,Home,-122.199,37.7618,NaN


In [30]:
persons = orca.get_table('persons').to_frame()

In [35]:
persons[persons.index.values == 2502445][['job_id', 'TOD', 'HW_ST']]

,job_id,TOD,HW_ST
person_id,,,
2502445,780221.0,NaN,NaN


In [24]:
num_persons_w_plans = len(plans['personId'].unique())

In [25]:
print("Mandatory trips assigned to {0} of {1} workers with jobs ({2}%) during simulation.".format(
    num_persons_w_plans, num_persons_w_jobs,
    np.round(num_persons_w_mode/num_persons_w_jobs * 100, 1)))

Mandatory trips assigned to 2578046 of 2578046 workers with jobs (100.0%) during simulation.


### Archive input data

In [35]:
orca.write_tables('./data/urbansim_beam_input_data.h5', [
    'persons', 'households', 'jobs', 'parcels', 'buildings', 'units'
])